# Agent-based model Quickstart guide

# Basic ABM model

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from abm.model import *
from abm.plotting import *
from abm.parallel import *
from abm.population import *
from abm.characteristics import load_population_dataset
from optimization.operators import *


## Running the simulation

### Running a basic simulation

We must first set up the model's parameters, using a dictionary:

In [2]:
def prepare_calendars(sol2):
    """Prepares the calendars for the simulation and stores it in "data/abm/vaud/prepared/temp_period_activities"
    sol2 : list of scenarios numbers for each segment

    :return: Number of periods for each activity
    """
    #Merge csv files for each segmantation according to the solution
    segments = ['0 - 9', '10 - 19', '20 - 29', '30 - 39', '40 - 49', '50 - 59', '60 - 69', '70 - 79', '80+']
    for i in range(len(sol2)):
        folder = "data/abm/vaud/prepared/scenarios/scenario_" + str(sol2[i]) + "/"
        file = folder + "vaud_period_activities_" + segments[i] + ".csv.gz"
        temp = pd.read_csv(file)
        if i == 0:
            df = temp
        else: 
            df = pd.concat([df, temp])
    
    unique_periods = df['period'].unique()
    activities = df['type'].unique()
    # Count the number of periods for each activity
    count_activities = dict()
    for activity in activities:
        count_activities[activity] = len(df[df['type'] == activity])

    #Applies preprocessing and saves the restriction schecule
    _AGENTS_ID_TRANSLATIONS_FILE_ = "data/abm/vaud/prepared/vaud_agents_id_translations.csv.gz"
    _FACILITIES_ID_TRANSLATIONS_FILE_ = "data/abm/vaud/prepared/vaud_facilities_id_translations.csv.gz"
    _PERIOD_ACTIVITIES_REP_ = "data/abm/vaud/prepared/temp_period_activities/"
    
    agents_translations = pd.read_csv(_AGENTS_ID_TRANSLATIONS_FILE_, index_col=0)
    agents_translations['agent_index']=agents_translations['agent_index'].astype(int)
    facilities_translations = pd.read_csv(_FACILITIES_ID_TRANSLATIONS_FILE_, index_col=0)
    facilities_translations['facility_index']=facilities_translations['facility_index'].astype(int)
    

    for period_index, period in enumerate(unique_periods):
        #print("Processing period ", period)
        # Isolate the activities that occurred during that period
        sub_activ = df[df['period'] == period]
        # Translate the agent ids to agent index
        sub_activ = sub_activ.merge(agents_translations, left_on="id", right_index=True)
        # Translate the facility names to indexes
        sub_activ = sub_activ.merge(facilities_translations, left_on="facility", right_index=True)
        # Only keep the relevant info
        sub_activ = sub_activ.drop(['age', 'period', 'id', 'facility'], axis=1)
        # Save the sub dataset
        sub_activ.to_csv(os.path.join(_PERIOD_ACTIVITIES_REP_, f"{str(period_index)}.csv.gz"),
                        index=False)
        
    return count_activities

In [3]:
params = {
        'inf_params': {'age': 0.000},
        'test_params': {'age': 0.000},
        'inf_fraction_param': 30,
        'inf_lvl_error_term': -15,
        'inf_proba_sigmoid_slope': 1.0,
        'test_inf_lvl_param': 0.1,
        'test_error_term': -15,
        'test_proba_sigmoid_slope': 10.0,
        'recovery_mean_time': 8.0,
        'recovery_std_time': 2.0,
        'Restriction_begin': 0,
        'Restriction_end': 60}


costs = dict({'home': 0, 'shop': 2, 'leisure': 1.5, 'work': 17.2, 'other': 2, 'education': 17.2})
normal_gdp = 62000000
N = 814000
lengthsimulation = 60
initial_infections = (np.arange(8)+1) * 100
n_scenarios = 4
average_remaining_working_years = 20 # Number of years to work
proportion_work_infection = 0.4 # Proportion of the cost of infection that is due to the loss of productivity
proportion_remoteless = 0.50
reduction_efficiency_remote_work = 0.2
cost_healthcare = 4700 # Cost of healthcare per day
severeforms = 0.1 #Pourcentage of severe forms in the positive tests

In [4]:
#solc = [0, 60, 1, 1, 1, 4, 1, 3, 1, 1, 1] #solution 1
#solc = [1, 59, 1, 1, 1, 1, 1, 1, 1, 4, 1] #solution 2
#solc =[2, 59, 1, 1, 1, 1, 4, 1, 1, 1, 1] #solution 3
solc =[2, 59, 1, 1, 1, 1, 4, 1, 1, 2, 1] #solution 4

sol2 = list(solc[2:])



#Call preprocessing function to obtain the restriction schedule according to the solution and count the number of period per activity
restriction_count_activities = prepare_calendars(sol2)
#Initializes the simulation and forces initial infections
abm = ABM(params)
abm.force_simulation_start(initial_infections)
# Runs the simulation with restriction period
abm.set_param('Restriction_begin', solc[0])
abm.set_param('Restriction_end', solc[1] )
abm.force_simulation_start(initial_infections)
abm.run_simulation(lengthsimulation, verbose=True)

_PERIOD_ACTIVITIES_ = 'data/abm/vaud/prepared/vaud_period_activities.csv.gz'    
df = pd.read_csv(_PERIOD_ACTIVITIES_, index_col=0)
activities = df['type'].unique()    
count_activities = dict()
for activity in activities:
    count_activities[activity] = len(df[df['type'] == activity])

        
# Compute I and D
results = abm.results.get_daily_results()
I = results["daily summed new infections"] # daily new infections
D = results["daily summed new infections"] # We should replace by number of death 
totalI= I.sum()
totalI.round(0)
totalD= D.sum()
totalD.round(2)

# Duration of the policy
Deltat = solc[1]-solc[0] 
# Number of confined activities
count_confined_activities = dict()
C_policies = 0
for activity in count_activities.keys():
    if activity in restriction_count_activities.keys():
        count_confined_activities[activity] = Deltat*(count_activities[activity] - restriction_count_activities[activity])
        C_policies += count_confined_activities[activity]*costs[activity]
    else:
        pass
C_policies = proportion_remoteless * reduction_efficiency_remote_work * C_policies

#Loss
sanitary_cost = (severeforms * totalD + totalI).round(2)
C_death = severeforms * totalD * normal_gdp / N *average_remaining_working_years 
C_infection = proportion_work_infection * normal_gdp / N / 365 * totalI * 5 # 5 is the average number of days of work lost per infection - should adapt with the model
C_helthcare = cost_healthcare * severeforms * totalI 
economic_cost = C_policies#(C_death + C_infection + C_helthcare + C_policies).round(2)/1000

objectives = [sanitary_cost, economic_cost]

print("EVALUATE")
print(objectives)

Loading activity data...
Processing period 0
Processing period 1
Processing period 2
Processing period 3
Processing period 4
Processing period 5
Processing period 6
Processing period 7
Processing period 8
Processing period 9
Processing period 10
Processing period 11
Processing period 12
Processing period 13
Processing period 14
Processing period 15
Processing period 16
Processing period 17
Processing period 18
Processing period 19
Processing period 20
Processing period 21
Processing period 22
Processing period 23
Loading reduction schedules ...
Loading activity data...
Processing period 0
Processing period 1
Processing period 2
Processing period 3
Processing period 4
Processing period 5
Processing period 6
Processing period 7
Processing period 8
Processing period 9
Processing period 10
Processing period 11
Processing period 12
Processing period 13
Processing period 14
Processing period 15
Processing period 16
Processing period 17
Processing period 18
Processing period 19
Processing per

/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:250: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Day  10
Day  11
Day  12
Day  13
Day  14
Day  15
Day  16
Day  17
Day  18
Day  19
Day  20
Day  21
Day  22
Day  23
Day  24
Day  25
Day  26
Day  27
Day  28
Day  29
Day  30
Day  31
Day  32
Day  33
Day  34
Day  35
Day  36
Day  37
Day  38
Day  39
Day  40
Day  41
Day  42
Day  43
Day  44
Day  45
Day  46
Day  47
Day  48
Day  49
Day  50
Day  51
Day  52
Day  53
Day  54
Day  55
Day  56
Day  57
Day  58
Day  59
Day  60
Simulation ended. 
EVALUATE
[308239.8, 66268160.67000001]


In [5]:
results = abm.results.get_daily_results()
results.head(115)

,daily summed new infections,daily summed infected agents,daily summed recovered agents,daily summed tests,daily summed positive tests,daily avg new infections,daily avg infected agents,daily avg recovered agents,daily avg tests,daily avg positive tests,day
day,,,,,,,,,,,
0,102,1272,0,0,0,4.250000,53.000000,0.000000,0.0,0.0,0
1,197,4899,0,0,0,8.208333,204.125000,0.000000,0.0,0.0,1
2,318,11149,0,0,0,13.250000,464.541667,0.000000,0.0,0.0,2
3,398,19741,0,0,0,16.583333,822.541667,0.000000,0.0,0.0,3
4,503,30693,21,0,0,20.958333,1278.875000,0.875000,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
64,3,3117,6721256,0,0,0.125000,129.875000,280052.333333,0.0,0.0,64
65,4,2380,6722043,0,0,0.166667,99.166667,280085.125000,0.0,0.0,65
66,13,1976,6722681,0,0,0.541667,82.333333,280111.708333,0.0,0.0,66


In [6]:
abm.results.get_per_period_results().head()

,new infections,infected agents,recovered agents,tests,positive tests,period,day
0,6,6,0,0,0,0,0
1,2,8,0,0,0,1,0
2,4,12,0,0,0,2,0
3,6,18,0,0,0,3,0
4,3,21,0,0,0,4,0


In [7]:
plotter = ABM_Plotter(abm)
# Displays the new infections and positive tests, and saves the image.
plotter.plot_curves(show_fig=True, save_img_to="tests/figures/simulation.jpg")

In [8]:
population_dataset = load_population_dataset()
plotter.plot_infection_spread(population_dataset, show_fig=True)

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Mapbox: 'bounds'

Did you mean "domain"?

    Valid properties:
        accesstoken
            Sets the mapbox access token to be used for this mapbox
            map. Alternatively, the mapbox access token can be set
            in the configuration options under `mapboxAccessToken`.
            Note that accessToken are only required when `style`
            (e.g with values : basic, streets, outdoors, light,
            dark, satellite, satellite-streets ) and/or a layout
            layer references the Mapbox server.
        bearing
            Sets the bearing angle of the map in degrees counter-
            clockwise from North (mapbox.bearing).
        center
            :class:`plotly.graph_objects.layout.mapbox.Center`
            instance or dict with compatible properties
        domain
            :class:`plotly.graph_objects.layout.mapbox.Domain`
            instance or dict with compatibl